# recount2 model building with CLAMP and PLIER

💡 **Environment:** `clamp-analyses`  

This notebook builds latent variable models (CLAMP and PLIER) from recount2 RNA‑seq RPKM data: it loads configuration and libraries, downloads and reads preprocessed recount2 data, maps Ensembl IDs to HGNC symbols and subsets genes and samples, stores the expression matrix in a bigstatsr FBM and applies filtering and z‑scoring (preprocessCLAMPFBM), computes SVD (big_randomSVD) and infers CLAMP K, runs CLAMPbase and CLAMPfull with pathway priors and runs PLIER with matched priors for comparison, and saves model matrices (B, Z).

# CLAMP

## Load libraries

In [1]:
# Create a timestamp to track the start of the analysis
start_time <- Sys.time()
cat("recount2 CLAMP and PLIER analysis started at:", format(start_time), "\n")

recount2 CLAMP and PLIER analysis started at: 2025-11-25 10:04:14 


In [2]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER)
library(CLAMP)

source(here("config.R"))

set.seed(config$recount2$RANDOM_SVD_SEED)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/clamp-analyses

Loading required package: RColorBrewer

Loading required package: gplots


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: pheatmap

Loading required package: qvalue



## Output directory

In [ ]:
output_data_dir <- config$recount2$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

## Settings

In [ ]:
block_size <- config$GENERAL$CHUNK_SIZE
N_CORES    <- config$recount2$N_CORES

## Download recount2 

In [ ]:
url      <- config$recount2$URL
out_dir  <- here::here("data", "recount2")
zip_file <- file.path(out_dir, "recount2.zip")
rds_file <- file.path(out_dir, "recount2_PLIER_data", "recount_data_prep_PLIER.RDS")
rpkm_file <- file.path(out_dir, "recount2_PLIER_data", "recount_rpkm.RDS")

if (!file.exists(rds_file) && !file.exists(rpkm_file)) {
    if (!dir.exists(out_dir)) dir.create(out_dir, recursive = TRUE)
    
    system(paste('wget', url, '-O', zip_file))
    unzip(zip_file, exdir = out_dir)
} else {
    message("Required files already exist. Skipping download and decompression.")
}

Required files already exist. Skipping download and decompression.



## Preprocess recount2 data

In [ ]:
# Read preprocessed recount2 data
data_mat <- readRDS(rds_file)

meta <- list()
meta$gene_symbols <- rownames(data_mat$rpkm.cm)
meta$samples <- colnames(data_mat$rpkm.cm)

rm(data_mat)

In [ ]:
# Read in recount2 file

rpkm.df <- readRDS(rpkm_file)

mart <- biomaRt::useDataset("hsapiens_gene_ensembl",
                            biomaRt::useMart("ensembl"))

genes <- unlist(lapply(strsplit(rpkm.df$ENSG, "[.]"), `[[`, 1))

rpkm.df$ensembl_gene_id <- unlist(lapply(strsplit(rpkm.df$ENSG, "[.]"),
                                         `[[`, 1))
                                         
gene.df <- biomaRt::getBM(filters = "ensembl_gene_id",
                          attributes = c("ensembl_gene_id", "hgnc_symbol"),
                          values = genes,
                          mart = mart)

gene.df <- gene.df %>% dplyr::filter(complete.cases(.))

rpkm.df <- dplyr::inner_join(gene.df, rpkm.df,
                             by = "ensembl_gene_id",
                             relationship = "many-to-many")

rownames(rpkm.df) <- make.names(rpkm.df$hgnc_symbol, unique = TRUE)

rpkm.df <- rpkm.df %>% dplyr::select(-c(ensembl_gene_id:ENSG))

data_mat <- rpkm.df[meta$gene_symbols, meta$samples]

rm(rpkm.df)

data_mat <- as.matrix(as.data.table(data_mat))

In [ ]:
fbm_files <- c(
  file.path(output_data_dir, "FBMrecount2.bk"),
  file.path(output_data_dir, "FBMrecount2_preproc.bk"),
  file.path(output_data_dir, "FBMrecount2_preproc_filtered.bk"),
  file.path(output_data_dir, "FBMrecount2_preproc_filtered.rds")
)

for (f in fbm_files) {
  if (file.exists(f)) {
    res <- unlink(f)
    if (identical(res, 0L)) {
      message("✅ Removed existing FBM file: ", f)
    } else {
      warning("⚠️ Failed to remove: ", f)
    }
  } else {
    message("ℹ️ Not found, skipping: ", f)
  }
}

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/recount2/FBMrecount2.bk

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/recount2/FBMrecount2.bk_preproc.bk

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/recount2/FBMrecount2.bk_preproc_filtered.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-analyses/output/recount2/FBMrecount2.bk_preproc_filtered.rds



In [ ]:
# Create the FBM
n_genes <- length(meta$gene_symbols)
n_samples <- length(meta$samples)

fbm_file <- file.path(output_data_dir, "FBMrecount2")
recount2FBM <- FBM(nrow = n_genes, ncol = n_samples, backingfile = fbm_file, create_bk = T)

In [ ]:
# Populate it with data
block_size <- config$GENERAL$CHUNK_SIZE
n_blocks <- ceiling(n_genes / block_size)

for (i in 1:n_blocks) {
  start_row <- (i-1) * block_size + 1
  end_row <- min(i * block_size, nrow(data_mat))
  recount2FBM[start_row:end_row, ] <- as.matrix(data_mat[start_row:end_row, ])
}

In [ ]:
# Preprocess and z‑score FBM
prep_recount2 <- preprocessCLAMPFBM(
  fbm        = recount2FBM,
  mean_cutoff= config$recount2$GENES_MEAN_CUTOFF,
  var_cutoff = config$recount2$GENES_VAR_CUTOFF
)

recount2_fbm_filt <- prep_recount2$fbm_filtered
recount2_rowStats <- prep_recount2$rowStats

Applying log2 transformation

Filling NAs with 0



In [ ]:
zscoreCLAMPFBM(recount2_fbm_filt, recount2_rowStats)

Applying Z-score transformation



In [ ]:
samples <- meta$samples
recount2_genes <- meta$gene_symbols[prep_recount2$kept_rows]

In [ ]:
saveRDS(samples, file = file.path(output_data_dir, "recount2_samples.rds"))
saveRDS(recount2_genes, file = file.path(output_data_dir, "recount2_genes.rds"))
saveRDS(recount2_fbm_filt, file = file.path(output_data_dir, "recount2_fbm_filt.rds"))

## SVD computation

In [ ]:
if (!file.exists(file.path(output_data_dir, "recount2_svdRes.rds"))) {

  g_fb <- nrow(recount2_fbm_filt)
  samples_fb <- ncol(recount2_fbm_filt)
  SVD_K_recount2 <- min(g_fb, samples_fb) - 1

  message("Using SVD K = ", SVD_K_recount2)

  if (N_CORES > 1) {
    # disable BLAS parallelization when parallelizing
    options(bigstatsr.check.parallel.blas = FALSE)
    blas_nproc <- getOption("default.nproc.blas")
    options(default.nproc.blas = NULL)
  }

  recount2_svdRes <- big_randomSVD(
    recount2_fbm_filt,
    k = SVD_K_recount2,
    ncores = N_CORES
  )

  saveRDS(recount2_svdRes, file = file.path(output_data_dir, "recount2_svdRes.rds"))

  if (N_CORES > 1) {
    # restore previous state
    options(bigstatsr.check.parallel.blas = TRUE)
    options(default.nproc.blas = blas_nproc)
  }

} else {
  message("recount2_svdRes already exists, skipping SVD computation.")
}

Using SVD K = 5999



## Estimate K for CLAMP

In [ ]:
CLAMP_K_recount2 <- num.pc(list(d = recount2_svdRes$d)) * 2
message("Inferred CLAMP K = ", CLAMP_K_recount2)

saveRDS(CLAMP_K_recount2, file = file.path(output_data_dir, "CLAMP_K_recount2.rds"))

Inferred CLAMP K = 832



## CLAMPbase

In [ ]:
recount2_baseRes <- CLAMPbase(
  Y      = recount2_fbm_filt,
  svdres = recount2_svdRes,
  trace  = TRUE,
  clamp_k = CLAMP_K_recount2
)

****

CLAMP k is set to 832

L1 is set to 35.6888738143438

L2 is set to 107.066621443032

Progress 1 / 200 | Bdiff=0.459785, minCor=0.702589

Progress 2 / 200 | Bdiff=0.035444, minCor=0.932119

Progress 3 / 200 | Bdiff=0.013512, minCor=0.974439

Progress 4 / 200 | Bdiff=0.008806, minCor=0.984172

Progress 5 / 200 | Bdiff=0.006394, minCor=0.989010

Progress 6 / 200 | Bdiff=0.004889, minCor=0.992503

Progress 7 / 200 | Bdiff=0.003897, minCor=0.994383

Progress 8 / 200 | Bdiff=0.003210, minCor=0.995265

Progress 9 / 200 | Bdiff=0.002715, minCor=0.995975

Progress 10 / 200 | Bdiff=0.002342, minCor=0.996665

Progress 11 / 200 | Bdiff=0.002053, minCor=0.997100

Progress 12 / 200 | Bdiff=0.001821, minCor=0.997500

Progress 13 / 200 | Bdiff=0.001631, minCor=0.997689

Progress 14 / 200 | Bdiff=0.001472, minCor=0.998018

Progress 15 / 200 | Bdiff=0.001339, minCor=0.998277

Progress 16 / 200 | Bdiff=0.001226, minCor=0.998497

Progress 17 / 200 | Bdiff=0.001128, minCor=0.998655

Progress 18 / 200

In [ ]:
recount2_baseRes$Z <- data.frame(recount2_baseRes$Z)
rownames(recount2_baseRes$Z) <- recount2_genes

recount2_baseRes$B <- data.frame(recount2_baseRes$B)
colnames(recount2_baseRes$B) <- samples

saveRDS(recount2_baseRes, file = file.path(output_data_dir, "CLAMPbase.rds"))

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV823,LV824,LV825,LV826,LV827,LV828,LV829,LV830,LV831,LV832
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GAS6,1.656654,0,0,0,0.0000000,0.8503073,0.2803847,0,0.6404271,0.2422511,⋯,0,0.2229221,0,0,0,0,0,0,0,0
MMP14,1.490046,0,0,0,0.0000000,0.4899336,0.0000000,0,0.4989182,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
DSP,1.642300,0,0,0,0.0000000,0.4149664,0.4115462,0,0.2766311,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
MARCKSL1,2.899502,0,0,0,0.3422124,0.0000000,0.0000000,0,0.0000000,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
SPARC,1.991356,0,0,0,0.0000000,0.5767266,0.0000000,0,0.4730226,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
CTSD,2.906868,0,0,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0


In [ ]:
model_dir <- file.path(output_data_dir, "CLAMPbase")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- recount2_baseRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- recount2_baseRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

## Prepare pathway priors

In [ ]:
recount2_gmtList <- list(
  KEGG = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=KEGG_2021_Human"),
  BP = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
  GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023")
)

# prefix each gene‐set name with its library to guarantee uniqueness
for(lib in names(recount2_gmtList)) {
  names(recount2_gmtList[[lib]]) <- paste0(lib, "_", names(recount2_gmtList[[lib]]))
}

recount2_pathMat <- gmtListToSparseMat(recount2_gmtList)
recount2_matched <- getMatchedPathwayMat(recount2_pathMat, recount2_genes)
recount2_chatObj <- getChat(recount2_matched)

Auto-detected name: KEGG_2021_Human

Using cached file for KEGG_2021_Human

Auto-detected name: GO_Biological_Process_2025

Using cached file for GO_Biological_Process_2025

Auto-detected name: GTEx_Tissues_V8_2023

Using cached file for GTEx_Tissues_V8_2023

There are 5844 genes in the intersection between data and prior

Removing 3296 pathways

Inverting...

done



## CLAMPfull

In [ ]:
recount2_fullRes <- CLAMPfull(
    Y = recount2_fbm_filt,
    svdres = recount2_svdRes,
    priorMat = recount2_matched,
    clamp.base.result = recount2_baseRes,
    use_cpp = TRUE,
    trace = TRUE,
    clamp_k = CLAMP_K_recount2
  )

** CLAMPfull **

using provided CLAMPbase result

CLAMP k is set to 832

L1=35.6888738143438; L2=107.066621443032

Progress 1 / 30 | Bdiff=0.000458

Progress 2 / 30 | Bdiff=0.007594

Progress 3 / 30 | Bdiff=0.007064

Estimated total runtime: ~6.6 min

Progress 4 / 30 | Bdiff=0.007791

Progress 5 / 30 | Bdiff=0.009431

Progress 6 / 30 | Bdiff=0.010059

Progress 7 / 30 | Bdiff=0.011233

Progress 8 / 30 | Bdiff=0.014777

Progress 9 / 30 | Bdiff=0.012048

Progress 10 / 30 | Bdiff=0.011607

Progress 11 / 30 | Bdiff=0.018083

Progress 12 / 30 | Bdiff=0.011340

Progress 13 / 30 | Bdiff=0.010272

Progress 14 / 30 | Bdiff=0.019521

Progress 15 / 30 | Bdiff=0.011078

Progress 16 / 30 | Bdiff=0.009438

Progress 17 / 30 | Bdiff=0.018238

Progress 18 / 30 | Bdiff=0.010508

Progress 19 / 30 | Bdiff=0.009156

Progress 20 / 30 | Bdiff=0.019579

Progress 21 / 30 | Bdiff=0.008802

Progress 22 / 30 | Bdiff=0.007460

Progress 23 / 30 | Bdiff=0.017314

Progress 24 / 30 | Bdiff=0.008235

Progress 25 / 30 | 

In [ ]:
recount2_baseRes$Z <- data.frame(recount2_baseRes$Z)
rownames(recount2_baseRes$Z) <- recount2_genes
head(recount2_baseRes$Z)

recount2_baseRes$B <- data.frame(recount2_baseRes$B)
colnames(recount2_baseRes$B) <- samples
head(recount2_baseRes$B)

recount2_fullRes$summary <- recount2_fullRes$summary %>%
    dplyr::rename(LV = LV_index)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

saveRDS(recount2_fullRes, file = file.path(output_data_dir, "CLAMPfull.rds"))

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV823,LV824,LV825,LV826,LV827,LV828,LV829,LV830,LV831,LV832
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GAS6,1.656654,0,0,0,0.0000000,0.8503073,0.2803847,0,0.6404271,0.2422511,⋯,0,0.2229221,0,0,0,0,0,0,0,0
MMP14,1.490046,0,0,0,0.0000000,0.4899336,0.0000000,0,0.4989182,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
DSP,1.642300,0,0,0,0.0000000,0.4149664,0.4115462,0,0.2766311,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
MARCKSL1,2.899502,0,0,0,0.3422124,0.0000000,0.0000000,0,0.0000000,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
SPARC,1.991356,0,0,0,0.0000000,0.5767266,0.0000000,0,0.4730226,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0
CTSD,2.906868,0,0,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0.0000000,⋯,0,0.0000000,0,0,0,0,0,0,0,0


,SRP000599.SRR013549,SRP000599.SRR013550,SRP000599.SRR013551,SRP000599.SRR013552,SRP000599.SRR013553,SRP000599.SRR013554,SRP000599.SRR013555,SRP000599.SRR013556,SRP000599.SRR013557,SRP000599.SRR013558,⋯,SRP035599.SRR1139372,SRP035599.SRR1139393,SRP035599.SRR1139388,SRP035599.SRR1139378,SRP035599.SRR1139399,SRP035599.SRR1139386,SRP035599.SRR1139375,SRP035599.SRR1139382,SRP035599.SRR1139356,SRP035599.SRR1139370
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
LV1,-0.867384901,-0.879844435,-0.873107420,-0.87148566,-0.8752239706,-0.875151653,-0.82379528,-8.266395e-01,-0.82067562,-0.82243867,⋯,-0.52053021,-0.51392282,-0.5257386,-0.5168343,-0.5285092,-0.5401691,-0.5359329,-0.5174164,-0.5040608,-0.5383226
LV2,-0.133615298,-0.127031401,-0.128278993,-0.13233526,-0.1323391474,-0.131399397,-0.16725299,-1.608620e-01,-0.16321947,-0.16428624,⋯,0.20624147,0.08443934,0.1507070,0.1775285,0.1510824,0.1872756,0.1662790,0.1755454,0.2382462,0.1948014
LV3,-0.033435328,-0.025611776,-0.023630533,-0.01513276,-0.0113543157,-0.024335924,-0.03823130,-3.274484e-02,-0.04298509,-0.04094039,⋯,-0.17209869,-0.16298852,-0.1731993,-0.1882488,-0.1720055,-0.1600504,-0.1761666,-0.1949913,-0.1505919,-0.1967838
LV4,-0.243989908,-0.235868650,-0.244113406,-0.24443976,-0.2327881755,-0.236407836,-0.21912488,-2.010877e-01,-0.22754310,-0.21997184,⋯,-0.09571376,-0.14086539,-0.2011386,-0.1156240,-0.2145210,-0.1652276,-0.2586825,-0.1696389,-0.1651986,-0.3765665
LV5,-0.019908724,-0.018702933,-0.008825051,-0.02083118,-0.0228384570,-0.017638770,-0.04534587,-4.036835e-02,-0.03258473,-0.03866460,⋯,-0.14597781,-0.14888982,-0.1407341,-0.1453838,-0.1405794,-0.1426756,-0.1224374,-0.1401182,-0.1452800,-0.1319524
LV6,-0.006614441,-0.007498693,-0.008060589,-0.01517550,0.0009615722,-0.008850218,-0.01707047,-7.413137e-05,-0.02293963,-0.01355327,⋯,-0.24519268,-0.24951182,-0.2527177,-0.2627610,-0.2605949,-0.2633666,-0.2431627,-0.2478865,-0.2298372,-0.2663158


In [ ]:
model_dir <- file.path(output_data_dir, "CLAMPfull")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- recount2_fullRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- recount2_fullRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- recount2_fullRes$summary
write.csv(summary, file.path(model_dir, "summary.csv"))

In [ ]:
# Create a timestamp to track the end of the analysis
end_time <- Sys.time()
elapsed_time <- end_time - start_time
cat("Recount2 CLAMP analysis completed at:", format(end_time), "\n")
cat("Total elapsed time:", format(elapsed_time), "\n")

Recount2 CLAMP analysis completed at: 2025-11-25 10:01:19 
Total elapsed time: 21.08286 hours 


# PLIER

Run PLIER with the same inputs

In [ ]:
recount2_plier = PLIER::PLIER(
    recount2_fbm_filt[], 
    as.matrix(recount2_matched), 
    svdres = recount2_svdRes, 
    Chat = as.matrix(recount2_chatObj), 
    doCrossval = TRUE, 
    k = CLAMP_K_recount2
  )

Removing 0 pathways with too few genes



[1] 107.0666
[1] "L2 is set to 107.066621443032"
[1] "L1 is set to 53.5333107215158"


errorY (SVD based:best possible) = 0.692



In [ ]:
colnames(gtex_plier$Z) <- paste0('LV', seq_len(ncol(gtex_plier$Z)))

gtex_plier$summary <- gtex_plier$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

colnames(gtex_plier$B) <- samples

saveRDS(gtex_plier, file = file.path(output_data_dir, "PLIER.rds"))

LV 1,3.038562e-11,-1.164697e-13,1.499369e-11,2.420048e-11,1.076099e-11,2.400729e-11,2.450469e-11,2.529211e-11,2.492415e-11,2.411303e-11,⋯,-2.252487e-14,-9.868426e-13,7.459374e-13,-1.238694e-13,-5.471264e-13,3.208637e-13,-1.203242e-12,-9.133111e-13,-7.087071e-13,-7.032934e-13
"2,Cells - Cultured Fibroblasts Female 70-79 Up",-3.281172e-02,-3.819620e-02,-4.021169e-02,-4.102633e-02,-3.925897e-02,-3.957653e-02,-9.111154e-03,-7.944571e-03,-1.111526e-02,-5.022869e-03,⋯,3.445032e-02,9.405888e-02,5.244139e-02,9.127825e-02,8.897180e-02,6.534399e-02,7.464888e-02,1.037498e-01,4.380141e-02,9.150614e-02
"3,mRNA Splicing, via Spliceosome (GO:0000398)",-9.481710e-02,-7.740678e-02,-9.260198e-02,-8.277387e-02,-7.081893e-02,-8.502113e-02,-6.194781e-02,-6.563719e-02,-6.688297e-02,-7.041583e-02,⋯,-5.538873e-02,-6.185108e-02,-3.681471e-02,-3.960329e-02,-6.117308e-02,-2.970421e-02,-8.491269e-02,-3.287660e-02,-2.841848e-02,-1.169766e-01
"4,Translation (GO:0006412)",-1.167416e-01,-1.169561e-01,-1.241410e-01,-1.144214e-01,-1.081250e-01,-1.183666e-01,-1.348015e-01,-1.444428e-01,-1.385527e-01,-1.426251e-01,⋯,5.650564e-02,2.639246e-02,1.109185e-01,6.937041e-02,8.791064e-02,1.055854e-01,1.084018e-01,7.893540e-02,7.554681e-02,1.749875e-01
"5,Double-Strand Break Repair via Homologous Recombination (GO:0000724)",-1.475776e-02,-2.136897e-02,-2.183746e-02,-1.868801e-02,-1.880394e-02,-1.911732e-02,-2.733048e-02,-3.572729e-02,-3.150585e-02,-2.717683e-02,⋯,4.954903e-03,2.586807e-02,2.185875e-02,4.275850e-02,4.369622e-02,5.298591e-02,-1.144091e-02,5.217841e-02,-1.841314e-02,2.890525e-02
LV 6,1.137411e-02,1.100171e-02,2.021871e-02,2.901634e-02,1.232140e-02,1.205028e-02,9.152861e-03,1.060361e-02,-6.413729e-04,6.352040e-03,⋯,-7.302503e-02,-5.922457e-02,-7.198972e-02,-5.402172e-02,-6.231897e-02,-6.095302e-02,-5.806830e-02,-6.060223e-02,-7.126057e-02,-5.891329e-02


In [ ]:
model_dir <- file.path(output_data_dir, "PLIER")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_plier$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_plier$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- gtex_plier$summary
write.csv(summary, file.path(model_dir, "summary.csv"))